# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

id — уникальный идентификатор скважины;
f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
product — объём запасов в скважине (тыс. баррелей).

## Загрузка и подготовка данных

In [ ]:
import pandas as pd
import pandas_profiling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from scipy import stats as st
import numpy as np
from sklearn.metrics import mean_squared_error
 

In [ ]:
reg_one = pd.read_csv('/datasets/geo_data_0.csv')
reg_two = pd.read_csv('/datasets/geo_data_1.csv')
reg_three = pd.read_csv('/datasets/geo_data_2.csv') #выгружаем данные

In [ ]:
reg_one.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [ ]:
reg_two.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [ ]:
reg_three.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [ ]:
reg_one.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [ ]:
reg_one.isna().sum() #смотрим кол-во пропусков в каждом датасете

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [ ]:
reg_two.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [ ]:
reg_two.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [ ]:
reg_three.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [ ]:
reg_three.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [ ]:
reg_one['id'].duplicated().sum()

10

In [ ]:
reg_one['id'].drop_duplicates()

0        txEyH
1        2acmU
2        409Wp
3        iJLyR
4        Xdl7t
         ...  
99995    DLsed
99996    QKivN
99997    3rnvd
99998    7kl59
99999    1CWhH
Name: id, Length: 99990, dtype: object

In [ ]:
reg_two['id'].duplicated().sum()

4

In [ ]:
reg_two['id'].drop_duplicates()

0        kBEdx
1        62mP7
2        vyE1P
3        KcrkZ
4        AHL4O
         ...  
99995    QywKC
99996    ptvty
99997    09gWa
99998    rqwUm
99999    relB0
Name: id, Length: 99996, dtype: object

In [ ]:
reg_three['id'].duplicated().sum()

4

In [ ]:
reg_three['id'].drop_duplicates()

0        fwXo0
1        WJtFt
2        ovLUW
3        q6cA6
4        WPMUX
         ...  
99995    4GxBu
99996    YKFjq
99997    tKPY3
99998    nmxp2
99999    V9kWn
Name: id, Length: 99996, dtype: object

In [ ]:
reg_one.drop(columns=['id'], axis =1, inplace= True)

In [ ]:
reg_two.drop(columns=['id'], axis =1, inplace= True)

In [ ]:
reg_three.drop(columns=['id'], axis =1, inplace= True)

In [ ]:
reg_one.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.1 MB


Сделали выгрузку данных, пропусков в данных нет. Избавились от дубликатов в колонке id  и удалили ее  в каждом датасете, т.к. для обучения модели не понадобится.

### Определим целевой признак

In [ ]:
reg_one_features = reg_one.drop(['product'], axis = 1)
reg_one_target = reg_one['product']
reg_two_features = reg_two.drop(['product'], axis = 1)
reg_two_target = reg_two['product']
reg_three_features = reg_three.drop(['product'], axis = 1)
reg_three_target = reg_three['product']

## Обучение и проверка модели

### Разделим выборки на обучающую и валидационную

In [ ]:
one_features_train, one_features_valid, one_target_train, one_target_valid = train_test_split(reg_one_features,
                                                                                             reg_one_target,
                                                                                             test_size = 0.25,
                                                                                             random_state = 12345)

two_features_train, two_features_valid, two_target_train, two_target_valid = train_test_split(reg_two_features,
                                                                                             reg_two_target,
                                                                                             test_size = 0.25,
                                                                                             random_state = 12345)

three_features_train, three_features_valid, three_target_train, three_target_valid = train_test_split(reg_three_features,
                                                                                                     reg_three_target,
                                                                                                     test_size = 0.25,
                                                                                                     random_state = 12345)

In [ ]:
model = LinearRegression()

### Обучаем модель на данных 1-го региона

In [ ]:
model.fit(one_features_train, one_target_train)
predictions_one = model.predict(one_features_valid)
mse_one = mean_squared_error(one_target_valid, predictions_one)
rmse_one = mse_one**0.5
one_mean = predictions_one.mean()
print(f'Средний запас предсказанного сырья: {one_mean}, RMSE = {rmse_one}')

Средний запас предсказанного сырья: 92.59256778438035, RMSE = 37.5794217150813


### Обучаем модель на данных 2-го региона

In [ ]:
model = LinearRegression()

In [ ]:
model.fit(two_features_train, two_target_train)
predictions_two = model.predict(two_features_valid)
mse_two = mean_squared_error(two_target_valid, predictions_two)
rmse_two = mse_two**0.5
two_mean = predictions_two.mean()
print(f'Средний запас предсказанного сырья: {two_mean}, RMSE = {rmse_two}')

Средний запас предсказанного сырья: 68.728546895446, RMSE = 0.893099286775617


### Обучаем модель на данных 3-го региона

In [ ]:
model.fit(three_features_train, three_target_train)
predictions_three = model.predict(three_features_valid)
mse_three = mean_squared_error(three_target_valid, predictions_three)
rmse_three = mse_three**0.5
three_mean = predictions_three.mean()
print(f'Средний запас предсказанного сырья: {three_mean}, RMSE = {rmse_three}')

Средний запас предсказанного сырья: 94.96504596800489, RMSE = 40.02970873393434


Мы обучили модель на данных трех регионов:

Регион 1: Средний запас предсказанного сырья: 92.59256778438035, RMSE = 37.5794217150813

Регион 2: Средний запас предсказанного сырья: 68.728546895446, RMSE = 0.893099286775617

Регион 3: Средний запас предсказанного сырья: 94.96504596800489, RMSE = 40.02970873393434

Средний запас предсказанного сырья больше всего у 1-го и 3-го региона, среднеквадратичная ошибка у 1-го региона меньше

## Подготовка к расчёту прибыли

### Определяем необходимые для расчета константы

In [ ]:
reg_budget = 10000000000 #Бюджет на разработку скважин в каждом регионе
one_barel_income = 450 #доход с одного барелля
income_per_unit = 450000   # доход с каждой единицы продукта
mines = 200 # количество наилучших шахт для разработки

### Считаем достаточный объем сырья для разработки

In [ ]:
reg_develop = reg_budget/income_per_unit
volume = reg_develop/mines
print(round(volume, 2))

111.11


### Сравниваем достаточный объем добычи со средним объемом предсказанного сырья

In [ ]:
mean_pred_volume = [one_mean, two_mean, three_mean]
regions = [1,2,3]
def volume_comp(mean_pred_volume):
    for i in range(len(mean_pred_volume)):
        if mean_pred_volume[i] < volume:
            print('Средний предсказанный объем в регионе', regions[i], 'меньше достаточного объема на'
                  , round(volume - mean_pred_volume[i],2 ))
        else:
            print('Средний предсказанный объем в регионе', regions[i], 'больше достаточного объема на',
                  round(mean_pred_volume[i] - volume,2))
volume_comp(mean_pred_volume)            
    

Средний предсказанный объем в регионе 1 меньше достаточного объема на 18.52
Средний предсказанный объем в регионе 2 меньше достаточного объема на 42.38
Средний предсказанный объем в регионе 3 меньше достаточного объема на 16.15


Мы расчитали достаточный объем сырья для разработки.
Сравнили  средние предсказанные запасы сырья в каждом регионе с достаточным объем добычи сырья для разработки, во всех трех регионах запасы меньше, чем нужно для безубыточной разработки. Меньше всего разница с достаточным объемом у 3-го региона, самая большая у 2-го региона

## Напишите функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:

### Напишем функцию, которая выбирает скважины с максимальными значениями предсказаний и просуммируем целевые значения по этим предсказания

In [ ]:
def revenue(target,predictions):
    target = pd.Series(target).reset_index(drop = True)
    predictions = pd.Series(predictions).reset_index(drop = True)
    max_predictions = predictions.sort_values(ascending = False)[:mines].index
    selected = target[max_predictions]
    revenue = selected.sum()*income_per_unit - reg_budget
    return(revenue)


### Считаем прибыль для полученного объема сырья

In [ ]:
print(revenue(one_target_valid, predictions_one))   
print(revenue(two_target_valid, predictions_two))
print(revenue(three_target_valid, predictions_three))

3320826043.1398506
2415086696.681511
2710349963.5998325


##  Посчитайте риски и прибыль для каждого региона:

In [ ]:
def boot(target,probabilities):
    state = np.random.RandomState(12345)
    values = []
    for i in range(1000):
        target_subsample = target.sample(n= 500, random_state=state, replace = True)
        probs_subsample = probabilities[target_subsample.index]
        values.append(revenue(target_subsample, probs_subsample))
    
    values = pd.Series(values)
    mean_values = values.mean()
    lower= values.quantile(0.025)
    upper = values.quantile(0.975)
    confidence_interval = st.t.interval(0.95, len(values)-1, loc=values.mean(), scale = values.sem())
    risk = (values < 0).mean()
    return round(mean_values,2), confidence_interval, round(risk*100, 2)

In [ ]:
one_mean_values, one_interval, one_risk = boot(one_target_valid.reset_index(drop=True), predictions_one)
print('Регион №1')
print('Средняя прибыль в регионе :', one_mean_values)
print('Доверительный интервал в регионе:', one_interval)
print('Риск убытков', one_risk,'%')

Регион №1
Средняя прибыль в регионе : 396164984.8
Доверительный интервал в регионе: (379620315.1479725, 412709654.45676965)
Риск убытков 6.9 %


In [ ]:
two_mean_values, two_interval, two_risk = boot(two_target_valid.reset_index(drop=True), predictions_two)
print('Регион №2')
print('Средняя прибыль в регионе :', two_mean_values)
print('Доверительный интервал в регионе:', two_interval)
print('Риск убытков', two_risk)

Регион №2
Средняя прибыль в регионе : 456045105.79
Доверительный интервал в регионе: (443147248.66390055, 468942962.909421)
Риск убытков 1.5


In [ ]:
three_mean_values, three_interval, three_risk = boot(three_target_valid.reset_index(drop=True), predictions_three)
print('Регион №3')
print('Средняя прибыль в регионе :', three_mean_values)
print('Доверительный интервал в регионе:', three_interval)
print('Риск убытков', three_risk,'%')

Регион №1
Средняя прибыль в регионе : 404403866.57
Доверительный интервал в регионе: (387445797.4712804, 421361935.6654332)
Риск убытков 7.6 %


Исходя из результатов самым рентабельным регионом оказался регион №2, средняя выручка составляет 456045105.79 млн.руб, риск убытков меньше всего - 1,5%. 

### Общий вывод

Общий вывод: Мы проанализировали данные по добыче нефти по трем регионам. Построили и обучили модель линейной регрессии, сделали предсказания на валидационной выборке, и получили средний предсказанный объем по добыче. Затем сравнили объем добычи нефти для безубыточной разработки и сравнили его со средним предсказанным объемом, преимущество оказалось у региона №3. Далее мы посчитали прибыль и риск убытков на выбранных скважинах по каждому региону и получили, что самым перспективным регионом является третий регион со средней выручкой в 456045105.79 млн.руб и 1.5% риском убытков. Нефтяной компании следует обратить свое внимание на данный регион и инвестировать в него.